In [28]:
from sklearn.ensemble import StackingClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
 
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.tree import plot_tree
from sklearn.metrics import roc_auc_score,accuracy_score,log_loss
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier


In [29]:
glass = pd.read_csv('Glass.csv')



In [30]:
glass.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,building_windows_float_processed
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,building_windows_float_processed
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,building_windows_float_processed
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,building_windows_float_processed
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,building_windows_float_processed


In [31]:
X = glass.drop('Type',axis=1)
y = glass['Type']

In [32]:
X.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0


In [33]:
y.head()

0    building_windows_float_processed
1    building_windows_float_processed
2    building_windows_float_processed
3    building_windows_float_processed
4    building_windows_float_processed
Name: Type, dtype: object

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=24, test_size=0.3,stratify=y)


In [35]:
knn = KNeighborsClassifier()
nb = GaussianNB()
dt = DecisionTreeClassifier(random_state=24)
svm = SVC(probability=True,random_state=24)
lr = LogisticRegression(random_state=24)
rf = RandomForestClassifier(random_state=24)

stack = StackingClassifier(estimators=[('KNN',knn),('NB',nb),('DT',dt),('SVM',svm)]  , final_estimator = lr,passthrough=True)

stack.fit(X_train,y_train)
y_pred = stack.predict(X_test)
print(accuracy_score(y_test,y_pred))

y_pred_proba = stack.predict_proba(X_test)
print(log_loss(y_test,y_pred_proba))

0.7076923076923077
0.7747800872138771


C:\anaconda-install\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [36]:
knn = KNeighborsClassifier()
nb = GaussianNB()
dt = DecisionTreeClassifier(random_state=24)
svm = SVC(probability=True,random_state=24)
rf = RandomForestClassifier(random_state=24)

stack = StackingClassifier(estimators=[('KNN',knn),('NB',nb),('DT',dt),('SVM',svm)]  , final_estimator = rf)

stack.fit(X_train,y_train)
y_pred = stack.predict(X_test)
print(accuracy_score(y_test,y_pred))

y_pred_proba = stack.predict_proba(X_test)
print(log_loss(y_test,y_pred_proba))

0.7076923076923077
1.252898468303256


### GridSearchCV

In [37]:
kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=24)
rf = RandomForestClassifier(random_state=24)
stack = StackingClassifier(estimators=[('KNN',knn),('NB',nb),('DT',dt),('SVM',svm)]  , final_estimator = rf)

In [38]:
stack.get_params()

{'cv': None,
 'estimators': [('KNN', KNeighborsClassifier()),
  ('NB', GaussianNB()),
  ('DT', DecisionTreeClassifier(random_state=24)),
  ('SVM', SVC(probability=True, random_state=24))],
 'final_estimator__bootstrap': True,
 'final_estimator__ccp_alpha': 0.0,
 'final_estimator__class_weight': None,
 'final_estimator__criterion': 'gini',
 'final_estimator__max_depth': None,
 'final_estimator__max_features': 'sqrt',
 'final_estimator__max_leaf_nodes': None,
 'final_estimator__max_samples': None,
 'final_estimator__min_impurity_decrease': 0.0,
 'final_estimator__min_samples_leaf': 1,
 'final_estimator__min_samples_split': 2,
 'final_estimator__min_weight_fraction_leaf': 0.0,
 'final_estimator__monotonic_cst': None,
 'final_estimator__n_estimators': 100,
 'final_estimator__n_jobs': None,
 'final_estimator__oob_score': False,
 'final_estimator__random_state': 24,
 'final_estimator__verbose': 0,
 'final_estimator__warm_start': False,
 'final_estimator': RandomForestClassifier(random_state=

In [39]:
params= { 'final_estimator__max_depth': [3,4,5],
        'SVM__C':np.linspace(0.0001,3,5),
        'final_estimator__n_estimators':[10,50],
         'DT__max_depth':[None,2,4],
        'passthrough':[True,False]}

gcv = GridSearchCV(stack,param_grid=params,scoring='neg_log_loss',verbose=3)

In [40]:
gcv.fit(X,y)

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV 1/5] END DT__max_depth=None, SVM__C=0.0001, final_estimator__max_depth=3, final_estimator__n_estimators=10, passthrough=True;, score=-1.236 total time=   0.1s
[CV 2/5] END DT__max_depth=None, SVM__C=0.0001, final_estimator__max_depth=3, final_estimator__n_estimators=10, passthrough=True;, score=-0.944 total time=   0.1s
[CV 3/5] END DT__max_depth=None, SVM__C=0.0001, final_estimator__max_depth=3, final_estimator__n_estimators=10, passthrough=True;, score=-1.099 total time=   0.1s
[CV 4/5] END DT__max_depth=None, SVM__C=0.0001, final_estimator__max_depth=3, final_estimator__n_estimators=10, passthrough=True;, score=-0.992 total time=   0.1s
[CV 5/5] END DT__max_depth=None, SVM__C=0.0001, final_estimator__max_depth=3, final_estimator__n_estimators=10, passthrough=True;, score=-0.929 total time=   0.1s
[CV 1/5] END DT__max_depth=None, SVM__C=0.0001, final_estimator__max_depth=3, final_estimator__n_estimators=10, passthroug

GridSearchCV(estimator=StackingClassifier(estimators=[('KNN',
                                                       KNeighborsClassifier()),
                                                      ('NB', GaussianNB()),
                                                      ('DT',
                                                       DecisionTreeClassifier(random_state=24)),
                                                      ('SVM',
                                                       SVC(probability=True,
                                                           random_state=24))],
                                          final_estimator=RandomForestClassifier(random_state=24)),
             param_grid={'DT__max_depth': [None, 2, 4],
                         'SVM__C': array([1.000000e-04, 7.500750e-01, 1.500050e+00, 2.250025e+00,
       3.000000e+00]),
                         'final_estimator__max_depth': [3, 4, 5],
                         'final_estimator__n_estimators': [10, 50],
                         'passthrough': [True, False]},
             scoring='neg_log_loss', verbose=3)

In [41]:
print(gcv.best_params_)
print(gcv.best_score_)

{'DT__max_depth': 4, 'SVM__C': 0.7500749999999999, 'final_estimator__max_depth': 5, 'final_estimator__n_estimators': 50, 'passthrough': True}
-0.9306837995227696


### Serializing the fitted object

In [44]:
import pickle

In [46]:
best_stack = gcv.best_estimator_

In [47]:
pkfile = open('stacking_glass.pkl','wb')

In [49]:
pickle.dump(best_stack,pkfile)